采集声音

In [1]:
from unihiker import Audio
import time

audio = Audio() #实例化音频

print("环境音大小")
for i in range(30):
    print(audio.sound_level())
    time.sleep(0.1)
	
	
print("录音3秒")
audio.record('3s.wav', 3) #录音3秒，存到文件3s.wav中

print("开始录音")
audio.start_record('6s.wav') #后台开始录音，存到文件6s.wav中
print("等待6秒")
time.sleep(6) #等待6秒
audio.stop_record() #停止录音
print("停止录音")


环境音大小
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
录音3秒
开始录音
等待6秒
停止录音


In [ ]:
   pip install dashscope

In [19]:
pip install -U openai

  Using cached openai-1.52.0-py3-none-any.whl (386 kB)
  Using cached openai-1.51.2-py3-none-any.whl (383 kB)
  Using cached openai-1.51.1-py3-none-any.whl (383 kB)
  Using cached openai-1.51.0-py3-none-any.whl (383 kB)
  Using cached openai-1.50.2-py3-none-any.whl (382 kB)
  Using cached openai-1.50.1-py3-none-any.whl (378 kB)
  Using cached openai-1.50.0-py3-none-any.whl (378 kB)
  Using cached openai-1.49.0-py3-none-any.whl (378 kB)
  Using cached openai-1.48.0-py3-none-any.whl (376 kB)
  Using cached openai-1.47.1-py3-none-any.whl (375 kB)
  Using cached openai-1.47.0-py3-none-any.whl (375 kB)
  Using cached openai-1.46.1-py3-none-any.whl (375 kB)
  Using cached openai-1.46.0-py3-none-any.whl (375 kB)
  Using cached openai-1.45.1-py3-none-any.whl (374 kB)
  Using cached openai-1.45.0-py3-none-any.whl (374 kB)
  Using cached openai-1.44.1-py3-none-any.whl (373 kB)
  Using cached openai-1.44.0-py3-none-any.whl (367 kB)
  Using cached openai-1.43.1-py3-none-any.whl (365 kB)
  Using ca

In [36]:
   pip install webrtcvad

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 KB 5.4 kB/s eta 0:00:00a 0:00:01m
  Preparing metadata (setup.py) ... done
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp37-cp37m-linux_aarch64.whl size=78448 sha256=88803a23a4d79820df2e3b8284b0256ac1506dda02452bda9b4ac49f5c58e0e1
  Stored in directory: /root/.cache/pip/wheels/11/f9/67/a3158d131f57e1c0a7d8d966a707d4a2fb27567a4fe47723ad
Successfully built webrtcvad
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pyaudio
import dashscope
from dashscope.audio.asr import Recognition, RecognitionCallback, RecognitionResult

# 设置您的API密钥
dashscope.api_key = 'sk-7c04ee6f9432492bb344baa7a5c0162f'

class Callback(RecognitionCallback):
    def on_complete(self) -> None:

    def on_error(self, result: RecognitionResult) -> None:
        print(f"错误: {result}")

    def on_event(self, result: RecognitionResult) -> None:
        sentence = result.get_sentence()
        if sentence:
            print(f"识别结果: {sentence['text']}")

# 初始化音频流
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=16000,
                input=True,
                frames_per_buffer=1024)

# 创建识别对象
callback = Callback()
recognition = Recognition(model='paraformer-realtime-v2',
                          format='pcm',
                          sample_rate=16000,
                          callback=callback)

# 开始识别
recognition.start()

try:
    print("开始说话...")
    while True:
        audio_data = stream.read(1024)
        recognition.send_audio_frame(audio_data)
except KeyboardInterrupt:
    print("停止识别")
    recognition.stop()

# 关闭音频流
stream.stop_stream()
stream.close()
p.terminate()

IndentationError: expected an indented block (3213303927.py, line 11)

In [7]:
pip install --upgrade dashscope

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyaudio
import dashscope
from dashscope.audio.asr import Recognition, RecognitionCallback, RecognitionResult
import time

# 设置您的API密钥
dashscope.api_key = 'sk-7c04ee6f9432492bb344baa7a5c0162f'

class Callback(RecognitionCallback):
    def __init__(self):
        self.final_result = ""

    def on_complete(self) -> None:
        if self.final_result:
            print(f"最终识别结果: {self.final_result}")
        self.final_result = ""  # 重置结果，为下一次识别做准备

    def on_error(self, result: RecognitionResult) -> None:
        print(f"错误: {result}")

    def on_event(self, result: RecognitionResult) -> None:
        sentence = result.get_sentence()
        if sentence and sentence.get('end_time') is not None:
            # 只在句子结束时更新最终结果
            self.final_result = sentence['text']

def create_recognition():
    callback = Callback()
    recognition = Recognition(model='paraformer-realtime-v2',
                              format='pcm',
                              sample_rate=16000,
                              callback=callback)
    recognition.start()
    return recognition

# 初始化音频流
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=16000,
                input=True,
                frames_per_buffer=1024)

# 创建识别对象
recognition = create_recognition()

try:
    print("开始说话...")
    silence_threshold = 500  # 静音阈值
    silence_duration = 1.0  # 静音持续时间（秒）
    last_audio_time = time.time()

    while True:
        audio_data = stream.read(1024)
        if max(abs(int.from_bytes(audio_data[i:i+2], 'little', signed=True)) for i in range(0, len(audio_data), 2)) > silence_threshold:
            last_audio_time = time.time()
            recognition.send_audio_frame(audio_data)
        elif time.time() - last_audio_time > silence_duration:
            # 如果静音持续超过指定时间，停止当前识别并开始新的识别
            recognition.stop()
            recognition = create_recognition()
            print("检测到长时间静音，重新开始识别...")
        time.sleep(0.01)  # 短暂休眠以减少 CPU 使用

except KeyboardInterrupt:
    print("停止识别")
    recognition.stop()
except Exception as e:
    print(f"发生错误: {e}")
finally:
    # 关闭音频流
    stream.stop_stream()
    stream.close()
    p.terminate()

print("程序结束")

ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.hdmi
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.hdmi
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.modem
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.modem
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.phoneline
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.phoneline
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port


开始说话...
错误: {"status_code": 44, "request_id": "6a889337ad3b4949b5ff30d804b2af5b", "code": "NO_VALID_AUDIO_ERROR", "message": "NO_VALID_AUDIO_ERROR", "output": null, "usage": null}
检测到长时间静音，重新开始识别...
错误: {"status_code": 44, "request_id": "677d4867c06f41d69cfb0baa0dfcd1b6", "code": "NO_VALID_AUDIO_ERROR", "message": "NO_VALID_AUDIO_ERROR", "output": null, "usage": null}
检测到长时间静音，重新开始识别...
错误: {"status_code": 44, "request_id": "878c3211e59f4d099262f526f1e5457d", "code": "NO_VALID_AUDIO_ERROR", "message": "NO_VALID_AUDIO_ERROR", "output": null, "usage": null}
检测到长时间静音，重新开始识别...
错误: {"status_code": 44, "request_id": "b0c184af922c40949084846de6c6a107", "code": "NO_VALID_AUDIO_ERROR", "message": "NO_VALID_AUDIO_ERROR", "output": null, "usage": null}
检测到长时间静音，重新开始识别...
错误: {"status_code": 44, "request_id": "c3a53d2a3cc247c1a831aff718e7d268", "code": "NO_VALID_AUDIO_ERROR", "message": "NO_VALID_AUDIO_ERROR", "output": null, "usage": null}
检测到长时间静音，重新开始识别...
错误: {"status_code": 44, "request_id":

InvalidParameter: Speech recognition has stopped.

In [ ]:
调用阿里云语音API实现语音实时识别

In [6]:
import pyaudio
import dashscope
from dashscope.audio.asr import Recognition, RecognitionCallback, RecognitionResult
import time
import numpy as np
import ctypes

# ALSA 错误处理
ERROR_HANDLER_FUNC = ctypes.CFUNCTYPE(None, ctypes.c_char_p, ctypes.c_int, ctypes.c_char_p, ctypes.c_int, ctypes.c_char_p)

def py_error_handler(filename, line, function, err, fmt):
    pass

c_error_handler = ERROR_HANDLER_FUNC(py_error_handler)

try:
    asound = ctypes.cdll.LoadLibrary('libasound.so')
    asound.snd_lib_error_set_handler(c_error_handler)
except:
    pass

# 设置您的API密钥
dashscope.api_key = 'sk-7c04ee6f9432492bb344baa7a5c0162f'

class Callback(RecognitionCallback):
    def __init__(self):
        self.last_result = ""

    def on_complete(self) -> None:
        print("识别完成")

    def on_error(self, result: RecognitionResult) -> None:
        print(f"错误: {result}")

    def on_event(self, result: RecognitionResult) -> None:
        sentence = result.get_sentence()
        if sentence:
            current_result = sentence['text']
            if current_result != self.last_result:
                print(f"识别结果: {current_result}")
                self.last_result = current_result

def is_speech(audio_data, threshold=500):
    audio_array = np.frombuffer(audio_data, dtype=np.int16)
    return np.max(np.abs(audio_array)) > threshold

# 初始化音频流
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=16000,
                input=True,
                frames_per_buffer=1024)

# 创建识别对象
callback = Callback()
recognition = Recognition(model='paraformer-realtime-v2',
                          format='pcm',
                          sample_rate=16000,
                          callback=callback)

# 开始识别
recognition.start()

try:
    print("开始说话...")
    buffer = []
    silence_frames = 0
    max_silence_frames = 50  # 大约 1 秒的静音

    while True:
        audio_data = stream.read(1024, exception_on_overflow=False)
        if is_speech(audio_data):
            buffer.append(audio_data)
            silence_frames = 0
        else:
            silence_frames += 1

        if len(buffer) >= 10 or (buffer and silence_frames >= max_silence_frames):
            combined_audio = b''.join(buffer)
            recognition.send_audio_frame(combined_audio)
            buffer = []

        if silence_frames >= max_silence_frames:
            silence_frames = 0
            print("检测到静音，等待新的语音输入...")

        time.sleep(0.01)  # 短暂休眠以减少 CPU 使用

except KeyboardInterrupt:
    print("停止识别")
    recognition.stop()
except Exception as e:
    print(f"发生错误: {e}")
finally:
    # 关闭音频流
    stream.stop_stream()
    stream.close()
    p.terminate()

print("程序结束")

开始说话...
检测到静音，等待新的语音输入...
检测到静音，等待新的语音输入...
识别结果: 太笨
识别结果: 太笨盼
识别结果: 太笨盼望着
识别结果: 太笨，盼望着冬天
识别结果: 太笨，盼望着盼望来了
识别结果: 太笨，盼望着盼望来了春净的脚
识别结果: 太笨，盼望着盼望来了春净的脚步
识别结果: 盼望着盼望着，冬天来了，纯净的脚步且
识别结果: 盼望着，盼望着，冬天来了，纯净的脚步，一切
识别结果: 盼望着，盼望着，冬天来了，纯净的脚步，一切都
识别结果: 盼望着，盼望着，冬天来了，纯净的脚步，一切都像刚睡醒
识别结果: 盼望着，盼望着，冬天来了，纯净的脚步，一切都像刚睡醒的样
识别结果: 盼望着盼望着，冬天来了，纯净的脚步，一切都像刚睡醒的样子。谢
识别结果: 盼望着盼望着，冬天来了，纯净的脚步，一切都像刚睡醒的样子。
识别结果: 盼望着盼望着，冬天来了，纯净的脚步，一切都像刚睡醒的样子。欣欣
识别结果: 盼望着盼望着，冬天来了，纯净的脚步，一切都像刚睡醒的样子，欣欣然点
识别结果: 盼望着盼望着，冬天来了，纯净的脚步，一切都像刚睡醒的样子，欣欣然点山说
识别结果: 盼望着盼望着，冬天来了，纯净的脚步，一切都像刚睡醒的样子，欣欣然点山顺
识别结果: 盼望着盼望着，冬天来了，纯净的脚步，一切都像刚睡醒的样子，欣欣然点山顺起来了
识别结果: 盼望着盼望着，冬天来了，纯净的脚步，一切都像刚睡醒的样子，欣欣然点山顺起来了水
识别结果: 盼望着盼望着，冬天来了，纯净的脚步，一切都像刚睡醒的样子，欣欣然点。
识别结果: 山顺起来了，水涨起来，太阳
识别结果: 山顺起来了，水涨起来，太阳的脸
识别结果: 山顺起来了，水涨起来，太阳的脸红起来
识别结果: 山顺起来了，水涨起来，太阳的脸红起来。小凯从土
识别结果: 山顺起来了，水涨起来，太阳的脸红起来。小凯从土地里
识别结果: 山顺起来了，水涨起来，太阳的脸红起来。小凯从土地里钻
识别结果: 山顺起来了，水涨起来，太阳的脸红起来。小凯从土地里钻出来
识别结果: 山顺起来了，水涨起来，太阳的脸红起来。小凯从土地里钻出来，嫩嫩的
识别结果: 山顺起来了，水涨起来，太阳的脸红起来。
识别结果: 小凯从土地里钻出来，嫩嫩的硬绿的园
识别结果: 小凯从土地里钻出来，嫩嫩的硬绿的园子里，田
识别结果: 小凯从土地里钻出来，嫩嫩的